In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.pandas.set_option('display.max_rows',None)
pd.pandas.set_option('display.max_columns',None)

In [ ]:
train=pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train.head()

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
target=train['Category'].unique()
print(target)

In [ ]:
data_dict = {}
count = 1
for data in target:
    data_dict[data] = count
    count+=1
    train["Category"] = train["Category"].replace(data_dict)

In [ ]:
train.head(50)

In [ ]:
x=train['DayOfWeek'].unique()
print(x)

In [ ]:
data_week_dict={
    'Monday':1,
    'Tuesday':2,
    'Wednesday':3,
    'Thursday':4,
    'Friday':5,
    'Saturday':6,
    'Sunday':7
}

In [ ]:
train['DayOfWeek']=train['DayOfWeek'].replace(data_week_dict)

In [ ]:
test=pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test.head()

In [ ]:
z=test['Id']

In [ ]:
test.shape

In [ ]:
test['DayOfWeek']=test['DayOfWeek'].replace(data_week_dict)

In [ ]:
train.drop(['Resolution','Descript'],axis=1,inplace=True)
test.drop(['Id'],axis=1,inplace=True)


In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test_x=test.copy()
train_x=train.copy()

In [ ]:
dataset=pd.concat((train,test),axis=0)

In [ ]:
dataset.columns

### Dates

In [ ]:
dataset['Dates'].head()

In [ ]:
dataset['Dates']=pd.to_datetime(dataset['Dates'],format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [ ]:
dataset['Year']=dataset['Dates'].dt.year
dataset['Month']=dataset['Dates'].dt.month
dataset['Date']=dataset['Dates'].dt.day
dataset['Hour']=dataset['Dates'].dt.hour
dataset['Minutes']=dataset['Dates'].dt.minute

In [ ]:
dataset.head()

In [ ]:
dataset.drop(['Dates'],axis=1,inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset.isnull().sum()

### Numerical Features

In [ ]:
numerical_features=[feature for feature in dataset.columns if dataset[feature].dtypes!='O']
numerical_features

In [ ]:
dataset[numerical_features].head()

### Categorical Features

In [ ]:
categorical_features=[feature for feature in dataset.columns if dataset[feature].dtypes=='O']
categorical_features

In [ ]:
dataset[categorical_features].head()

In [ ]:
dataset[categorical_features].columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
for feature in ['PdDistrict', 'Address']:
    dataset[feature]=le.fit_transform(dataset[feature])

In [ ]:
dataset[categorical_features].head()

In [ ]:
dataset.head()

In [ ]:
core=dataset.corr()
core

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(core,annot=True)

In [ ]:
corr=dataset.corr()
print(corr['Category'])

In [ ]:
dataset.head()

In [ ]:
train=dataset.iloc[:878049,:]
train.head()

In [ ]:
train.shape

In [ ]:
test=dataset.iloc[878049:]
test.head()

In [ ]:
test.drop(['Category'],axis=1,inplace=True)

In [ ]:
test.head()

In [ ]:
test=pd.concat((z,test),axis=1)

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
X_train=train.drop(['Category'],axis=1)

In [ ]:
y_train=train['Category']

In [ ]:
X_test=test

In [ ]:
X_train['DayOfWeek'].unique()

In [ ]:
X_test.head(100)

### Model

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

### Initialize logistic Regression Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)

In [ ]:
X_test.head()

In [ ]:
X_test.drop(['Id'],axis=1,inplace=True)

In [ ]:
predictions=knn.predict(X_test)

In [ ]:
from collections import OrderedDict
data_dict_new = OrderedDict(sorted(data_dict.items()))
print(data_dict_new)

In [ ]:
predictions

In [ ]:
test=pd.concat((z,test),axis=1)

In [ ]:
result_dataframe = pd.DataFrame({
    "Id": test["Id"]
})
for key,value in data_dict_new.items():
    result_dataframe[key] = 0
count = 0
for item in predictions:
    for key,value in data_dict.items():
        if(value == item):
            result_dataframe[key][count] = 1
    count+=1
result_dataframe.to_csv("submission_knn.csv", index=False)
